# Getting and cleaning complaint problems data

First, data from the NYC Housing Complaint problems dataset are read into a pandas data frame.

In [1]:
import pandas as pd
import numpy as np
import re

### 

def get_problem_data():
    query = ("https://data.cityofnewyork.us/api/views/a2nx-4u46/rows.csv?accessType=DOWNLOAD")
    complaint_problems = pd.read_csv(query)
    return complaint_problems
            
complaint_problems = get_problem_data()
complaint_problems.head(3)

,ProblemID,ComplaintID,UnitTypeID,UnitType,SpaceTypeID,SpaceType,TypeID,Type,MajorCategoryID,MajorCategory,MinorCategoryID,MinorCategory,CodeID,Code,StatusID,Status,StatusDate,StatusDescription
0,14603248,6991025,91,APARTMENT,541,BATHROOM,1,EMERGENCY,9,PLUMBING,65,BASIN/SINK,624,DRAIN PIPE BLOCKED OR BROKEN,2,CLOSE,09/13/2014,The Department of Housing Preservation and Dev...
1,14643878,7007545,91,APARTMENT,543,ENTIRE APARTMENT,1,EMERGENCY,59,HEAT/HOT WATER,348,APARTMENT ONLY,2834,NO HOT WATER,2,CLOSE,09/03/2014,The Department of Housing Preservation and Dev...
2,14630215,7000457,91,APARTMENT,541,BATHROOM,3,NON EMERGENCY,65,WATER LEAK,381,SLOW LEAK,2831,AT WALL OR CEILING,2,CLOSE,10/24/2014,The Department of Housing Preservation and Dev...


Next, the dataset is cleaned as follows:
* The following features are dropped:
    ** UnitType, SpaceType, Type, MajorCategory, MinorCategory, Code, Status,

In [2]:
print complaint_problems.shape
complaint_problems = complaint_problems.drop(['UnitType', 'SpaceType', 'Type', 'MajorCategory','MinorCategory','Code','Status'],1)
print complaint_problems.shape

(852936, 18)
(852936, 11)


Then rows are dropped if:
    * StatusID == 1 (i.e. the record is open)
    * They are incomplete (i.e. only complete records are included in analytic sample)

In [3]:
complaint_problems = complaint_problems[(complaint_problems.StatusID == 2)]
print complaint_problems.shape
complaint_problems = complaint_problems[~(complaint_problems.isnull().any(axis=1))]
print complaint_problems.shape

(830437, 11)
(827515, 11)


Next, rows are dropped if:

* Values in fields UnitType, SpaceType, Type, MajorCategory, MinorCategory, and Code are not in codebook (implying the value is missing or mis-entered).

Sets of allowed code values are read in from txt files, which were produced from the HPD-producedd codebook 'HPD_Complaints_Open_Data.pdf' as follows:

* 'HPD_Complaints_Open_Data.pdf' was converted to an html file using the pdfminer package.
* All text from each section of the codebook was copied from this html page into a text file.
* This text file was manually cleaned and everything except allowed code values was deleted.

In [4]:
def get_allowed_codes_from_txt(feature_name):
    file_path = 'codebook_for_complaint_problems/' + str(feature_name) + '.txt'
    with open(file_path) as f:
        raw_codebook = f.readlines()
    code_values = []
    for line in raw_codebook:
        code_values.extend(re.findall('[0-9]+',line))
    code_values = np.array(code_values).astype(int)
    return code_values

allowed_UnitTypeID = get_allowed_codes_from_txt('UnitType')
allowed_TypeID = get_allowed_codes_from_txt('ProblemType')
allowed_SpaceTypeID = get_allowed_codes_from_txt('SpaceType')
allowed_MajorCategoryID = get_allowed_codes_from_txt('MajorCategory')
allowed_MinorCategoryID = get_allowed_codes_from_txt('MinorCategory')
allowed_CodeID = get_allowed_codes_from_txt('Code')

In [5]:
##Note there is an extra whitespace in the SpaceTypeID_ column name
print complaint_problems.columns

Index([u'ProblemID', u'ComplaintID', u'UnitTypeID', u'SpaceTypeID ', u'TypeID',
       u'MajorCategoryID', u'MinorCategoryID', u'CodeID', u'StatusID',
       u'StatusDate', u'StatusDescription'],
      dtype='object')


In [6]:
##so we need to first rename the columns
complaint_problems.columns = ['ProblemID', 'ComplaintID', 'UnitTypeID', 'SpaceTypeID', 'TypeID', 'MajorCategoryID', 'MinorCategoryID', 'CodeID', 'StatusID', 'StatusDate', 'StatusDescription']

In [7]:
print complaint_problems.shape
complaint_problems_proc = complaint_problems[(complaint_problems['SpaceTypeID'].isin(allowed_SpaceTypeID))]
print complaint_problems_proc.shape

complaint_problems_proc = complaint_problems_proc[(complaint_problems_proc['UnitTypeID'].isin(allowed_UnitTypeID))]
print complaint_problems_proc.shape

complaint_problems_proc = complaint_problems_proc[(complaint_problems_proc['TypeID'].isin(allowed_TypeID))]
print complaint_problems_proc.shape

complaint_problems_proc = complaint_problems_proc[(complaint_problems_proc['MajorCategoryID'].isin(allowed_MajorCategoryID))]
print complaint_problems_proc.shape

complaint_problems_proc = complaint_problems_proc[(complaint_problems_proc['MinorCategoryID'].isin(allowed_MinorCategoryID))]
print complaint_problems_proc.shape

complaint_problems_proc = complaint_problems_proc[(complaint_problems_proc['CodeID'].isin(allowed_CodeID))]
print complaint_problems_proc.shape

(827515, 11)
(787767, 11)
(787767, 11)
(787767, 11)
(787767, 11)
(787764, 11)
(777524, 11)


Then, StatusDescriptions are processed and coded using the scheme outlined below:

In [8]:
###Proposed classes for StatusDescription:

## 1. "not able to gain access" = r'not\sables\sto\sgain\saccess'
##1. "unable to access" = r'unable\sto\saccess'
## 2. "inspected the following conditions. No violations were issued." = r'inspected\sthe\sfollowing\sconditions\.\sNo\sviolations\swere\sissued'
##2. "Heat was not required at the time of the inspection. No violations were issued" = r'Heat\swas\snot\srequired\sat\sthe\stime\sof\sthe\sinspection\.\sNo\sviolations\swere\sissued'
## 3. ". Violations were issued" = r'\.Violations\swere\sissued'
##4. ". Violations were previously issued" = r'\.Violations\swere\spreviously\sissued'
##5. "conditions were corrected" = r'conditions\swere\scorrected'
##5. "advised by a tenant' [that heat or hot water was restored] = r'advised\sby\sa\stenant'
##6. "conditions are still open" = r'conditions\sare\still\open'
##7. "inspection to test the paint for lead" = r'inspection\sto\stest\sthe\spaint\sfor\slead'
##8. ". A Section 8 Failure was issued." = r'\.\sA\sSection\s8\sFailure\swas\sissued\.'
##0. Not one of the above.

def infer_complaint_status(input_string):
    try:
        input_string = str(input_string)
    except:
        print input_string
    if bool(re.search(r'not\sable\sto\sgain\saccess', input_string)):
        code = 1 
    elif bool(re.search(r'unable\sto\saccess', input_string)):
        code = 1
    elif bool(re.search(r'inspected\sthe\sfollowing\sconditions\.\sNo\sviolations\swere\sissued', input_string)):
        code = 2
    elif bool(re.search(r'Heat\swas\snot\srequired\sat\sthe\stime\sof\sthe\sinspection\.\sNo\sviolations\swere\sissued', input_string)):
        code = 2
    elif bool(re.search(r'\.\sViolations\swere\sissued', input_string)):
        code = 3
    elif bool(re.search(r'\.\sViolations\swere\spreviously\sissued', input_string)):
        code = 4
    elif bool(re.search(r'conditions\swere\scorrected', input_string)):
        code = 5 
    elif bool(re.search(r'advised\sby\sa\stenant', input_string)):
        code = 5
    elif bool(re.search(r'conditions\sare\sstill\sopen', input_string)):
        code = 6
    elif bool(re.search(r'inspection\sto\stest\sthe\spaint\sfor\slead', input_string)):
        code = 7
    elif bool(re.search(r'\.\sA\sSection\s8\sFailure\swas\sissued\.', input_string)):
        code = 8
    else:
        code = 0
    return code

complaint_problems_proc['StatusDescriptionID'] = complaint_problems_proc['StatusDescription'].map(infer_complaint_status)

In [9]:
print len(complaint_problems_proc[(complaint_problems_proc['StatusDescriptionID'] == 0)])

0


Since there are no records with StatusDescriptionID == 0, we know we've successfully handled every status description. As such, proceed by dropping StatusID (since they're all 2), and StatusDescription.

In [10]:
complaint_problems_proc = complaint_problems_proc.drop(['StatusID','StatusDescription'],axis=1)
print complaint_problems_proc.head(6)

   ProblemID  ComplaintID  UnitTypeID  SpaceTypeID  TypeID  MajorCategoryID  \
0   14603248      6991025          91          541       1                9   
1   14643878      7007545          91          543       1               59   
2   14630215      7000457          91          541       3               65   
3   14548958      6967900          91          541       1                9   
4   14548959      6967900          91          541       3                9   
5   14548960      6967900          91          543       3               58   

   MinorCategoryID  CodeID  StatusDate  StatusDescriptionID  
0               65     624  09/13/2014                    3  
1              348    2834  09/03/2014                    5  
2              381    2831  10/24/2014                    1  
3               63    2538  07/29/2014                    3  
4               63    2540  08/04/2014                    1  
5              343    2691  08/04/2014                    1  


Finally, we check that we only have records from 2014 and beyond- once we've confirmed this, we're done!

In [11]:
print complaint_problems_proc.shape
complaint_problems_proc.StatusDate = pd.to_datetime(complaint_problems_proc.StatusDate)
print complaint_problems_proc[(complaint_problems_proc.StatusDate.map(lambda x: x.year == 2014 or x.year == 2015))].shape

(777524, 10)
(777524, 10)


Complaint problems: Date range: 2014-2015 Only closed complaints (exclude “conditions are still open” complaints)
Read in housing maintenance code complaints dataset (HMCC)(only records with status dates after 2014, and statusid = 2), including only the following features: -complaintID -boroughID -block -lot
Construct the BBL using the Borough, block, and lot features in the housing

In [ ]:
import pandas as pd
import numpy as np

#API calls
def get_data():
    '''
    Returns DataFrame with housing maintenance code complaints dataset (only records with status dates after 2014, 
    and statusid = 2)
    '''
    query = ("https://data.cityofnewyork.us/resource/jspd-2hr7.json?statusid=2"
             "&$where=statusdate%20between%20'2014-01-01T00:00:00'%20and%20'2015-10-01T00:00:00'"
             "&$limit=1000")
    hmc_complaints = pd.read_json(query)
    i = 1 
    more_data_available = True
    while more_data_available:
        query = ("https://data.cityofnewyork.us/resource/jspd-2hr7.json?statusid=2"
                 "&$where=statusdate%20between%20'2014-01-01T00:00:00'%20and%20'2015-10-01T00:00:00'"
                 "&$limit=1000&$order=:id&$offset=" + str(i*1000))
        more_data = pd.read_json(query)
        hmc_complaints = pd.concat([hmc_complaints, more_data],ignore_index=True)
        print 'Currently have ', len(hmc_complaints), ' records'
        i += 1
        
        if len(more_data) < 1000:
            more_data_available = False
    return pd.DataFrame(hmc_complaints)


#API approach
data = get_data()
print data.shape.head(5)

In [18]:
#Standart approach

data = pd.read_csv("Data_Housing_maintenance_code_complaints.csv")

In [19]:
def make_BBL(borough, block, lot): 
    '''
    The borough code is one numeric digit. 
    The tax block is one to five numeric digits, preceded with leading zeros 
    when the block is less than five digits.
    The tax lot is one to four digits and is preceded with leading zeros
    when the lot is less than four digits.
    
    >>> make_BBL(1,16,100)
    1000160100
    >>> make_BBL(3,15828,7501)
    3158287501
    '''
    return int(str(borough) + str(block).zfill(5) + str(lot).zfill(4))
    
data = data[['ComplaintID', 'BoroughID', 'Block', 'Lot']]
data['BBL'] = map(make_BBL, data['BoroughID'], data['Block'], data['Lot'])
print data.head(n = 5)

   ComplaintID  BoroughID  Block  Lot         BBL
0      6960137          1   1904    4  1019040004
1      6960832          1   1918    4  1019180004
2      6946867          1   1083    1  1010830001
3      6966946          1   2073   29  1020730029
4      6963755          1   1869   44  1018690044


In [20]:
data.shape

(428506, 5)

Check the number of unique ComplaintIDs. 
There is a mismatch for unique values on this field between HMCC(428506 unique) and HPDCP(410133 unique)

In [30]:
qq = complaint_problems_proc['ComplaintID']
qq = qq.unique()
len(qq)

410133

Merge matching ComplaintIDs on complaint_problems_proc and data (inner join)

In [31]:
result_hmcc_and_hpdcp = pd.merge(complaint_problems_proc, data, on='ComplaintID', how='inner')

In [ ]:
Shape after merging

In [33]:
result_hmcc_and_hpdcp.shape

(718547, 14)

In [34]:
result_hmcc_and_hpdcp.head(6)

,ProblemID,ComplaintID,UnitTypeID,SpaceTypeID,TypeID,MajorCategoryID,MinorCategoryID,CodeID,StatusDate,StatusDescriptionID,BoroughID,Block,Lot,BBL
0,14603248,6991025,91,541,1,9,65,624,2014-09-13,3,2,2794,15,2027940015
1,14603246,6991025,91,546,1,65,380,2828,2014-09-13,3,2,2794,15,2027940015
2,14603247,6991025,91,541,1,9,63,2538,2014-09-13,3,2,2794,15,2027940015
3,14643878,7007545,91,543,1,59,348,2834,2014-09-03,5,2,2809,31,2028090031
4,14630215,7000457,91,541,3,65,381,2831,2014-10-24,1,2,3739,55,2037390055
5,14548958,6967900,91,541,1,9,63,2538,2014-07-29,3,2,2841,14,2028410014


Count the number of occurrences for ComplaintID = 6991025

In [ ]:
gg = (complaint_problems_proc['ComplaintID'] == 6991025).sum()
gg